# WALK THE TALK

## Bibliotecas utilizadas

In [18]:
# Bibliotecas padrão do Python
import os
import base64
from datetime import datetime
import json

# Bibliotecas de terceiros
import xml.etree.ElementTree as ET
from io import StringIO
import zipfile
import requests
from requests import Session
import pandas as pd
from pymongo import MongoClient
from dotenv import load_dotenv
import fitz
from openai import OpenAI

## Funções

### Busca o ZIP da CVM

In [19]:
""" 
Busca o arquivo ZIP no repositório da CVM e salva-o em pasta dedicada no diretório corrente
Retorna True caso o download tenha sido bem sucedido e False caso contrário
Retorna o caminho do arquivo ZIP salvo
"""

def busca_zip_cvm(ano):
    
    # URL do repositório da CVM
    url_repo = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/FRE/DADOS/'

    # URL do arquivo ZIP
    nome_zip_cvm = f'fre_cia_aberta_{ano}.zip'
    url_zip_cvm = url_repo + nome_zip_cvm

    # Define o caminho onde o arquivo ZIP será salvo
    caminho_local = os.path.join(os.getcwd(), 'arquivos_cvm\\')
    os.makedirs(caminho_local, exist_ok=True)

    # Faz o download do arquivo ZIP
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url_zip_cvm, headers=headers)

    if response.status_code == 200:
        caminho_zip = os.path.join(caminho_local, nome_zip_cvm)

        with open(caminho_zip, "wb") as file:
            file.write(response.content)
            
        print(f"ZIP {nome_zip_cvm} salvo com sucesso.")
        return True, caminho_zip
    else:
        print(f"Falha no download do ZIP {nome_zip_cvm}. Erro {response.status_code}.")
        return False, None
    
# Execução
#ano = datetime.datetime.now().year
ano = 2024
cvm_sucesso, caminho_zip = busca_zip_cvm(ano)

ZIP fre_cia_aberta_2024.zip salvo com sucesso.


### Lista conteúdo do ZIP

In [20]:
# Abre o ZIP e retorna em uma lista os arquivos contidos nele

def lista_arquivos_zip(nome_zip):

    # Abre o arquivo ZIP
    with zipfile.ZipFile(nome_zip, 'r') as zip_ref:

        # Lista os arquivos contidos no ZIP
        lista_arquivos = zip_ref.namelist()

    # Retorna a lista de arquivos
    print(f'Lista de arquivos recuperada com sucesso. De {lista_arquivos[0]} a {lista_arquivos[-1]}')
    return lista_arquivos

# Execução
lista_zip = lista_arquivos_zip(caminho_zip)

Lista de arquivos recuperada com sucesso. De fre_cia_aberta_2024.csv a fre_cia_aberta_volume_valor_mobiliario_2024.csv


### Extrai CSV do ZIP para dicionário

In [21]:
# Extrai os dados de um CSV específico dentro do ZIP
# Formata os rótulos das colunas e os tipos de dados
# Filtra os dados para recuperar apenas a versão mais recente de cada empresa
# Retorna uma lista de dicionários com os dados para obtenção do XML de cada empresa

def csv_para_dict(arquivo_zip, nome_arquivo):
    with zipfile.ZipFile(arquivo_zip, 'r') as zip_ref:
        with zip_ref.open(nome_arquivo) as file:
            try:
                df = pd.read_csv(file, encoding='cp1252', sep=';', decimal=',')
                print(f"Arquivo {nome_arquivo} carregado com sucesso.")
                
                # Padroniza os rótulos das colunas
                if nome_arquivo == f'fre_cia_aberta_{ano}.csv':
                    rotulos_colunas = ['CNPJ_Companhia', 'Data_Referencia', 'Versao', 'Nome_Companhia', 'Codigo_CVM', 'Categoria_Doc', 'ID_Documento', 'Data_Recebimento', 'Link_Doc'] 
                    df = df.rename(columns=dict(zip(df.columns, rotulos_colunas)))

                # Padroniza os tipos de dados                
                for column in df.columns:
                    if 'Data' in column:
                        df[column] = pd.to_datetime(df[column]).dt.date
                    if 'Preco' in column or 'Valor' in column or 'Quantidade' in column:
                        df[column] = pd.to_numeric(df[column], errors='coerce')

                # Filtra para manter apenas a versão mais recente de cada empresa
                idx = df.groupby('CNPJ_Companhia')['Versao'].transform('max') == df['Versao']
                df_recente = df[idx]

                # Converte o DataFrame filtrado em dicionário para inserção no MongoDB
                registros_recentes = df_recente.to_dict('records')
                return registros_recentes

            except UnicodeDecodeError as e:
                print(f"Erro de decodificação: {e}")
                return None

# Execução
nome_arquivo = lista_zip[0]
dados = csv_para_dict(caminho_zip, nome_arquivo)
os.remove(caminho_zip)
dados

Arquivo fre_cia_aberta_2024.csv carregado com sucesso.


[{'CNPJ_Companhia': '02.635.522/0001-95',
  'Data_Referencia': datetime.date(2024, 3, 31),
  'Versao': 9,
  'Nome_Companhia': 'JALLES MACHADO S.A.',
  'Codigo_CVM': 25496,
  'Categoria_Doc': 'FRE WEB',
  'ID_Documento': 133268,
  'Data_Recebimento': datetime.date(2023, 12, 22),
  'Link_Doc': 'http://www.rad.cvm.gov.br/ENETCONSULTA/frmDownloadDocumento.aspx?CodigoInstituicao=1&NumeroSequencialDocumento=133268'},
 {'CNPJ_Companhia': '06.981.381/0001-13',
  'Data_Referencia': datetime.date(2024, 3, 31),
  'Versao': 10,
  'Nome_Companhia': 'CTC - CENTRO DE TECNOLOGIA CANAVIEIRA S.A.',
  'Codigo_CVM': 23981,
  'Categoria_Doc': 'FRE WEB',
  'ID_Documento': 134110,
  'Data_Recebimento': datetime.date(2024, 2, 19),
  'Link_Doc': 'http://www.rad.cvm.gov.br/ENETCONSULTA/frmDownloadDocumento.aspx?CodigoInstituicao=1&NumeroSequencialDocumento=134110'},
 {'CNPJ_Companhia': '07.628.528/0001-59',
  'Data_Referencia': datetime.date(2024, 6, 30),
  'Versao': 3,
  'Nome_Companhia': 'BRASILAGRO - CIA BRA

### Baixa ZIP de cada relatório individual

In [5]:
""" 
Busca o arquivo ZIP no repositório da CVM e salva-o em pasta dedicada no diretório corrente
Retorna True caso o download tenha sido bem sucedido e False caso contrário
Retorna o caminho do arquivo ZIP salvo
"""

def busca_zip_relatorio(url, cod_cvm):

    # Substitui 'http' por 'https' na URL
    url = url.replace("http://", "https://")

    # Define o caminho onde o arquivo ZIP será salvo
    caminho_local = os.path.join(os.getcwd(), 'arquivos_cvm\\')
    os.makedirs(caminho_local, exist_ok=True)

    # Faz o download do arquivo ZIP
    headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "pt,en-US;q=0.9,en;q=0.8,es;q=0.7,pt-BR;q=0.6,es-419;q=0.5",
        "Cache-Control": "no-cache",
        "Connection": "keep-alive",
        "Cookie": "_ga_81XRXPNHSK=GS1.3.1708735940.1.0.1708735940.0.0.0; _ga_H8QM8DH015=GS1.3.1710252799.2.0.1710252799.0.0.0; _ga_6KR6GSRJZZ=GS1.1.1711483988.5.1.1711484321.0.0.0; ASP.NET_SessionId=uj05yychuwi0jnps00faitsb; dtCookie=v_4_srv_27_sn_8592F9AFAEEA6AAD845CED4F835CD528_perc_100000_ol_0_mul_1_app-3Aca883f3beebf9526_1_rcs-3Acss_0; BIGipServerpool_www.rad.cvm.gov.br_443=1242043402.47873.0000; TS01f82b11=011d592ce16f3b33e2abc77a55c56b8a376b8e4ddc8cb9eddd60e14cd95a007180a2568ad8bdc586cd2e2e8e1398e0604fe76936ad; _gid=GA1.3.1681611676.1711654202; _ga_146QLVPQ44=GS1.3.1711654202.22.1.1711654291.0.0.0; rxVisitor=1711654304222UEIST157USV3PR06THKGSEMQ5GL26BGA; dtSa=-; _ga_1HWVQMZJJT=GS1.1.1711654355.3.1.1711654668.0.0.0; _ga=GA1.1.1244918504.1708717704; TS01a5fb2d=016e3b076ff953e4dd5561ccaaf1b9a51c4c976453ab1a3db435bda15004f971c22e1965d9cad26173216e4a7837b44cd7ab6a09a5; TS01871345=016e3b076fa0140e142e3a3cf2dced5da19cbd02bb899d7e0eb84e0a762f4bce6a6ba695b0447db437022a47d5c475ff0a763d79a9; rxvt=1711656772915|1711654304227; dtPC=27$454972132_578h-vIPANPKQLJHFSAVGPTRCQAPHHESAKKSGR-0e0",
        "Host": "www.rad.cvm.gov.br",
        "Pragma": "no-cache",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.0",
        "sec-ch-ua": "\"Chromium\";v=\"122\", \"Not(A:Brand\";v=\"24\", \"Microsoft Edge\";v=\"122\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\""
    }

    # Utiliza sessions para manter a conexão aberta e evitar erros de timeout
    with Session() as session:
        session.headers.update(headers)
#        session.get('http://www.rad.cvm.gov.br/ENETCONSULTA/')
        response = session.get(url)

        # Se o download for bem sucedido, salva o arquivo ZIP
        if response.status_code == 200:
            caminho_zip = os.path.join(caminho_local, cod_cvm + '.zip')

            with open(caminho_zip, "wb") as file:
                file.write(response.content)
                
            print(f"Relatório ZIP da empresa {cod_cvm} salvo com sucesso em {caminho_zip}.")
            return True, caminho_zip
        # Se o download falhar, exibe uma mensagem de erro
        else:
            print(f"Falha no download do relatório ZIP da empresa {cod_cvm}. Erro {response.status_code}.")
            return False, None
    
# Execução
url_rel = dados[2]['Link_Doc']
codigo_cvm = str(dados[2]['Codigo_CVM'])
rel_zip_sucesso, caminho_rel_zip = busca_zip_relatorio(url_rel, codigo_cvm)

Relatório ZIP da empresa 2437 salvo com sucesso em c:\Users\paulo\OneDrive - Insper - Institudo de Ensino e Pesquisa\9 Projeto Final I\walkthetalk\arquivos_cvm\2437.zip.


### Extrai o XML do arquivo ZIP

In [6]:
"""
Extrai o arquivo XML de um ZIP
Retorna o caminho do arquivo XML extraído
O arquivo XML extraído é salvo em pasta dedicada no diretório corrente
"""

def extrai_xml(caminho_zip):
    
    # Define o caminho onde o arquivo ZIP será salvo
    caminho_local = os.path.join(os.getcwd(), 'arquivos_cvm\\')
    os.makedirs(caminho_local, exist_ok=True)

    # Abre o arquivo ZIP
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        # Lista os arquivos contidos no ZIP
        lista_arquivos = zip_ref.namelist()
        
        # Filtra para encontrar o arquivo XML de interesse
        arquivo_xml_interesse = [arquivo for arquivo in lista_arquivos if 'FRE' in arquivo][0]
        
        # Extrai o arquivo XML de interesse
        zip_ref.extract(arquivo_xml_interesse, caminho_local)

    # Caminho completo para o arquivo XML extraído
    arquivo_xml_path = os.path.join(caminho_local, arquivo_xml_interesse)
    print(f"Arquivo XML extraído com sucesso em {arquivo_xml_path}.")
    return arquivo_xml_path

# Execução
xml_path = extrai_xml(caminho_rel_zip)

# Exclui o arquivo ZIP após a extração do XML
# os.remove(caminho_rel_zip)

Arquivo XML extraído com sucesso em c:\Users\paulo\OneDrive - Insper - Institudo de Ensino e Pesquisa\9 Projeto Final I\walkthetalk\arquivos_cvm\002437FRE31-12-2023v21.xml.


### Obtém os dados de interesse a partir do XML

In [7]:
"""
Função para extrair dados e replicar a estrutura de um elemento XML para as tags especificadas
xml_path: caminho do arquivo XML a ser processado
tags_buscadas: Lista de tags que estamos buscando no XML
Retorna dicionário contendo os dados encontrados
"""

def extrair_dados(xml_path, tags_buscadas):

    # Abrir o arquivo com o encoding especificado
    with open(xml_path, 'r', encoding='cp1252') as file:
        # Parsear o arquivo XML
        tree = ET.parse(file)
        root = tree.getroot()

        # Dicionário para armazenar os dados extraídos
        dados = {}

        # Função recursiva para processar cada elemento
        def processar_elemento(el):
            # Se o elemento é uma das tags buscadas, captura tudo sob ele
            if el.tag in tags_buscadas:
                return {el.tag: capturar_estrutura_aninhada(el)}
            # Caso contrário, continua a busca nos elementos filhos
            else:
                for filho in el:
                    resultado = processar_elemento(filho)
                    if resultado:
                        # Se já existe a chave e é uma lista, adiciona. Se não, transforma em lista.
                        if filho.tag in dados:
                            if isinstance(dados[filho.tag], list):
                                dados[filho.tag].append(resultado[filho.tag])
                            else:
                                dados[filho.tag] = [dados[filho.tag], resultado[filho.tag]]
                        else:
                            dados.update(resultado)

        def capturar_estrutura_aninhada(el):
            # Se não tem filhos, retorna o texto do elemento
            if not list(el):
                return el.text.strip() if el.text else ''
            # Se tem filhos, cria um dicionário para sua estrutura aninhada
            subdados = {}
            for filho in el:
                resultado_filho = capturar_estrutura_aninhada(filho)
                # Mesmo tratamento para possíveis múltiplos filhos com a mesma tag
                if filho.tag in subdados:
                    if isinstance(subdados[filho.tag], list):
                        subdados[filho.tag].append(resultado_filho)
                    else:
                        subdados[filho.tag] = [subdados[filho.tag], resultado_filho]
                else:
                    subdados[filho.tag] = resultado_filho
            return subdados

        # Inicia o processamento a partir do elemento raiz
        resultado_raiz = processar_elemento(root)
        if resultado_raiz:
            dados.update(resultado_raiz)

        return dados

# Exemplo de uso
tags_buscadas = ['DadosEmpresa', 'VersaoDocumento', 'DataReferencia', 'DescricaoCaracteristicasOrgaosAdmECF', 'DescricaoRHEmissor']
dados_extraidos = extrair_dados(xml_path, tags_buscadas)

# Exibe os dados extraídos
print(dados_extraidos)

{'DadosEmpresa': {'CodigoCvm': '002437', 'RazaoSocialEmpresa': 'CENTRAIS ELET BRAS S.A. - ELETROBRAS', 'CnpjEmpresa': '00001180000126', 'TipoEmpresa': '1'}, 'VersaoDocumento': '21', 'DataReferencia': '31/12/2023', 'DescricaoCaracteristicasOrgaosAdmECF': {'DescricaoCorRaca': {'XmlFormularioReferenciaDadosFREFormularioAssembleiaGeralEAdmDescricaoCaracteristicasOrgaosAdmECFCorRaca': [{'OrgaoAdministracao': 'Diretoria', 'Amarelo': '0', 'Branco': '11', 'Preto': '0', 'Pardo': '0', 'Indigena': '0', 'Outros': '0', 'PrefereNaoResponder': '0', 'NaoSeAplica': 'false'}, {'OrgaoAdministracao': 'Conselho de Administração - Efetivos', 'Amarelo': '0', 'Branco': '9', 'Preto': '0', 'Pardo': '0', 'Indigena': '0', 'Outros': '0', 'PrefereNaoResponder': '0', 'NaoSeAplica': 'false'}, {'OrgaoAdministracao': 'Conselho de Administração - Suplentes', 'Amarelo': '0', 'Branco': '', 'Preto': '', 'Pardo': '', 'Indigena': '', 'Outros': '', 'PrefereNaoResponder': '0', 'NaoSeAplica': 'true'}, {'OrgaoAdministracao': 'Co

### Trata os dados extraídos do XML

In [8]:
"""
Função para ajustar os dados extraídos, convertendo tipos de dados e formatos
dados: Dicionário com os dados extraídos do XML
Retorna dicionário com os dados ajustados
Ajustes realizados:
- Converte strings numéricas para int ou float
- Converte datas para o formato datetime
- Substitui strings vazias por 0
- Ignora 'CodigoCvm' e 'CnpjEmpresa'
- Faz chamada recursiva para dicionários aninhados e processa listas
- Ignora campos que não puderem ser convertidos
"""

def ajustar_dados(dados):

    for chave, valor in dados.items():
        # Se o valor for um dicionário, faz a chamada recursiva
        if isinstance(valor, dict):
            dados[chave] = ajustar_dados(valor)
        # Se for uma lista, processa cada item da lista
        elif isinstance(valor, list):
            dados[chave] = [ajustar_dados(item) if isinstance(item, dict) else item for item in valor]
        else:
            # Converte para datetime se for uma data
            if chave == 'DataReferencia' and isinstance(valor, str) and valor:
                try:
                    dados[chave] = datetime.strptime(valor, '%d/%m/%Y').date()
                except ValueError:
                    pass
            # Converte strings numéricas para int ou float, mas ignora 'CodigoCvm' e 'CnpjEmpresa'
            elif chave not in ['CodigoCvm', 'CnpjEmpresa'] and isinstance(valor, str):
                if valor.isdigit():
                    dados[chave] = int(valor)
                else:
                    try:
                        dados[chave] = float(valor)
                    except ValueError:
                        # Trata campos vazios, substituindo por 0
                        dados[chave] = 0 if valor == '' else valor
    return dados

# Supondo que 'dados_extraidos' é o seu dicionário original
dados_ajustados = ajustar_dados(dados_extraidos)

dados_ajustados

{'DadosEmpresa': {'CodigoCvm': '002437',
  'RazaoSocialEmpresa': 'CENTRAIS ELET BRAS S.A. - ELETROBRAS',
  'CnpjEmpresa': '00001180000126',
  'TipoEmpresa': 1},
 'VersaoDocumento': 21,
 'DataReferencia': datetime.date(2023, 12, 31),
 'DescricaoCaracteristicasOrgaosAdmECF': {'DescricaoCorRaca': {'XmlFormularioReferenciaDadosFREFormularioAssembleiaGeralEAdmDescricaoCaracteristicasOrgaosAdmECFCorRaca': [{'OrgaoAdministracao': 'Diretoria',
     'Amarelo': 0,
     'Branco': 11,
     'Preto': 0,
     'Pardo': 0,
     'Indigena': 0,
     'Outros': 0,
     'PrefereNaoResponder': 0,
     'NaoSeAplica': 'false'},
    {'OrgaoAdministracao': 'Conselho de Administração - Efetivos',
     'Amarelo': 0,
     'Branco': 9,
     'Preto': 0,
     'Pardo': 0,
     'Indigena': 0,
     'Outros': 0,
     'PrefereNaoResponder': 0,
     'NaoSeAplica': 'false'},
    {'OrgaoAdministracao': 'Conselho de Administração - Suplentes',
     'Amarelo': 0,
     'Branco': 0,
     'Preto': 0,
     'Pardo': 0,
     'Indigen

### Extrai PDF do XML

In [9]:
"""
Função para extrair o PDF de um XML
caminho_xml: caminho do arquivo XML a ser processado
codigo_cvm: código CVM da empresa
tag_de_interesse: tag do XML que contém o PDF a ser extraído
Retorna True se o PDF foi extraído com sucesso e False caso contrário
Retorna o caminho do PDF extraído
"""

def extrair_pdf_de_xml(caminho_xml, codigo_cvm, tag_de_interesse):

    # Define o caminho onde o arquivo ZIP será salvo
    caminho_local = os.path.join(os.getcwd(), 'arquivos_cvm\\')
    os.makedirs(caminho_local, exist_ok=True)

    try:
        # Abrir e ler o arquivo XML com a codificação 'cp1252'
        with open(caminho_xml, 'r', encoding='cp1252') as file:
            xml_content = file.read()

        # Parsear o XML
        tree = ET.parse(StringIO(xml_content))
        root = tree.getroot()

        # Encontrar a tag de interesse e extrair os dados do PDF
        tag_interesse_el = root.find(f'.//{tag_de_interesse}')
        if tag_interesse_el is not None:
            nome_arquivo_pdf = tag_interesse_el.find('NomeArquivoPdf').text
            pdf_data = tag_interesse_el.find('ImagemObjetoArquivoPdf').text
            pdf_bytes = base64.b64decode(pdf_data)

            nome_arquivo_saida = os.path.join(caminho_local, codigo_cvm + f'_{tag_de_interesse}.pdf')
            with open(nome_arquivo_saida, 'wb') as pdf_file:
                pdf_file.write(pdf_bytes)
            print(f"PDF extraído e salvo como {nome_arquivo_saida}")
            return True, nome_arquivo_saida
        else:
            print(f"Elemento <{tag_de_interesse}> não encontrado.")
            return False, None
    except UnicodeDecodeError as e:
        print(f"Erro de decodificação ao tentar ler o arquivo XML: {e}")
        return False, None
    except ET.ParseError as e:
        print(f"Erro ao parsear o XML: {e}")
        return False, None

# Exemplo de uso da função
tag_de_interesse = 'InfoASG'
sucesso_pdf, caminho_pdf = extrair_pdf_de_xml(xml_path, codigo_cvm, tag_de_interesse)

PDF extraído e salvo como c:\Users\paulo\OneDrive - Insper - Institudo de Ensino e Pesquisa\9 Projeto Final I\walkthetalk\arquivos_cvm\2437_InfoASG.pdf


### Extrai texto do PDF

In [10]:
def pdf_para_string(caminho_pdf):

    # Abre o arquivo PDF
    pdf_document = fitz.open(caminho_pdf)

    # Inicializa a string de texto
    texto_pdf = ''

    # Itera sobre as páginas do PDF
    for pagina_num in range(pdf_document.page_count):
        pagina = pdf_document.load_page(pagina_num)
        texto_pdf += pagina.get_text()

    return texto_pdf

# Exemplo de uso
texto_pdf = pdf_para_string(caminho_pdf)
print(texto_pdf)

 
1.9. 
Em relação a informações ambientais, sociais e de governança corporativa (ASG), 
indicar:  
 
Jornada EESG 
 
A Eletrobras vem adotando práticas sustentáveis e inovadoras desde a sua fundação, há mais de 
61 anos. Contribui substancialmente para fazer a Matriz Elétrica Brasileira ser uma das mais 
limpas do mundo, dispondo de uma capacidade instalada formada por 97% de fontes de baixa 
emissão de carbono. 
 
Temos o propósito empresarial de colocar toda a nossa força para o desenvolvimento 
sustentável da sociedade. Há 17 anos somos signatários do Pacto Global das Nações Unidas, 
maior iniciativa universal de sustentabilidade empresarial e priorizamos 9 Objetivos de 
Desenvolvimento Sustentável, os Objetivos do Desenvolvimento Sustentável (“ODS”) da Agenda 
2030, sendo o nosso compromisso reconhecido como case de sucesso pela Rede Brasil do Pacto 
Global. Desde 2009, emitimos anualmente o Inventário de Gases de Efeito Estufa das empresas 
Eletrobras, seguindo metodologia do GHG

### Processa texto com LLM

In [17]:
def openai_chat(openai_api_key, prompt, texto_pdf):

    client = OpenAI(api_key = openai_api_key)

    chat = client.chat.completions.create(
            model="gpt-3.5-turbo", 
            temperature = 0,
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": texto_pdf}])

    json_str = chat.choices[0].message.content

    # Converter a string JSON em um dicionário Python
    json_dict = json.loads(json_str)
    return json_dict


# Execução
prompt = """
O seu papel nesta conversa é o de um analista especializado na pauta ESG a quem foi designada a tarefa de avaliar questionários respondidos por empresas de capital aberto. 

Tais questionários estão relacionados ao seguinte contexto: "A Resolução nº 59 da Comissão de Valores Mobiliários (CVM), que entrou em vigor em 2023, estabelece um novo regime de divulgação para o Formulário de Referência, principalmente no que diz respeito a informações sobre temáticas ambientais, sociais e de governança corporativa (ESG, ou ASG na sigla em Português). O modelo “pratique ou explique” adotado atende a uma demanda crescente de investidores quanto a transparência sobre tais temas." 

O texto enviado enviado contém partes de um questionário padrão relacionadas a esse contexto, incluindo tanto as perguntas quanto as respostas fornecidas pela empresa. Considere que a empresa pode responder cada pergunta separadamente ou todas em um só bloco. 

Baseado somente na leitura do texto enviado, responda as perguntas a seguir, retornando as suas respostas em formato JSON com as seguintes chaves: DIVULGA_ESG, URL_RELATORIO, MATRIZ_MAT, MATRIZ_INDICADORES, ODS, ODS_OBJETIVOS, TCFD, DIVULGA_GEE, ESCOPO_GEE, URL_GEE, DIVERSIDADE, MITIGACAO, EXPLIQUE.

PERGUNTAS:
1. A empresa divulga informações ASG em relatório anual ou em documento específico para esta finalidade? (responda apenas True ou False)
2. Qual a URL informada para o acesso do relatório ou documento? (responda com uma lista incluindo apenas URLs válidas; se não houver nenhuma, deixe a lista vazia)
3. Em seus relatórios, a empresa considera uma matriz de materialidade e indicadores-chave de desempenho ASG? (responda apenas True, False ou deixe vazio caso a empresa não responda) 
4. Caso a empresa considere uma matriz de materialidade, quais são os indicadores-chave de desempenho ASG mencionados? (responda com uma lista de indicadores; se não houver nenhum, deixe a lista vazia)
5. Em seus relatórios, a empresa leva em consideração os Objetivos de Desenvolvimento Sustentável (ODS) estabelecidos pela ONU? (responda apenas True, False ou deixe vazio caso a empresa não responda)
6. Caso a empresa considere os Objetivos de Desenvolvimento Sustentável (ODS), quais são os principais objetivos mencionados? (responda com uma lista de objetivos; se não houver nenhum, deixe a lista vazia)
7. Em seus relatórios, a empresa considera as recomendações da Força-Tarefa para Divulgações Financeiras Relacionadas às Mudanças Climáticas (TCFD)? (responda apenas True, False ou deixe vazio caso a empresa não responda)
8. A empresa realiza inventários de emissão de gases do efeito estufa? (responda apenas True, False ou deixe vazio caso a empresa não responda)
9. Caso a empresa realize inventários de emissão de gases do efeito estufa (GEE), qual o escopo dos inventários informado? (responda com uma lista de escopos; se não houver nenhum, deixe a lista vazia)
10. Caso a empresa divulgue inventários de emissão de gases do efeito estufa (GEE), qual a URL informada para o acesso a esses inventários? (responda com uma lista de URLs; se não houver nenhuma, deixe a lista vazia)
11. Caso haja menção a compromissos e metas relacionados a questões de Gênero, Raça e Diversidade, resuma em um parágrafo os principais compromissos da empresa com relação ao tema. Dê preferência a compromissos concretos e mensuráveis.
12. Caso haja menção a compromissos e metas relacionados à redução de emissões de GEE e mitigação das mudanças climáticas, resuma em um parágrafo os principais compromissos da empresa com relação ao tema. Dê preferência a compromissos concretos e mensuráveis.
13. Considerando o modelo "pratique ou explique" adotado pelo questionário e as respostas obtidas acima, resuma em um parágrafo as principais explicações fornecidas pela empresa.

Por fim, siga sempre as seguintes diretrizes ao elaborar a sua resposta:
1. Sua resposta deve ser precisa, completa e baseada em trechos específicos do questionário para verificar sua autenticidade.
2. Se você não tem certeza, simplesmente diga que não sabe, em vez de inventar uma resposta.
3. Seja cético em relação às informações divulgadas no relatório, reconhecendo que as informações fornecidas representam a visão da empresa e podem ser exageradamente otimistas.
4. Mantenha suas respostas em formato JSON com as chaves mencionadas acima.

"""

openai_api_key = os.getenv("OPENAI_API_KEY")

respostas_gpt = openai_chat(openai_api_key, prompt, texto_pdf)

respostas_gpt

{'DIVULGA_ESG': True,
 'URL_RELATORIO': ['https://eletrobras.com/pt/Paginas/Sustentabilidade.aspx'],
 'MATRIZ_MAT': True,
 'MATRIZ_INDICADORES': ['Ética, integridade e compliance',
  'Relacionamento com comunidades',
  'Saúde, bem-estar e segurança do trabalhador',
  'Mudanças climáticas',
  'Biodiversidade e serviços ecossistêmicos',
  'Transição e eficiência energética',
  'Atração, desenvolvimento e retenção de colaboradores',
  'Gestão de água e efluentes',
  'Direitos humanos',
  'Gestão de riscos e emergência',
  'Inovação e tecnologia'],
 'ODS': True,
 'ODS_OBJETIVOS': ['ODS 7 - Energia Acessível e Limpa',
  'ODS 13 - Combate às Alterações Climáticas',
  'ODS 8',
  'ODS 9',
  'ODS 10',
  'ODS 11',
  'ODS 12',
  'ODS 15',
  'ODS 16'],
 'TCFD': True,
 'DIVULGA_GEE': True,
 'ESCOPO_GEE': ['Escopo 1', 'Escopo 2', 'Escopo 3'],
 'URL_GEE': ['https://eletrobras.com/pt/Paginas/Estrategia-Climatica.aspx'],
 'DIVERSIDADE': 'A Eletrobras possui um Comitê de Gênero, Raça e Diversidade que p

## Script de execução

In [ ]:
"""
ALGORITMO:

PARA 2023 (precisa pensar na lógica de organização dos dados no MongoDB para 2024 não sobrepor 2023)
1. Busca ZIP na CVM
2. Lista arquivos dentro do ZIP
3. Extrai CSV do ZIP para dicionário
4. Para cada URL de relatório no CSV:
    4.1. Busca ZIP do relatório
    4.2. Extrai XML do ZIP do relatório
    4.3. Obtém dados do XML
    4.4. Ajusta dados do XML
    4.5. Extrai PDF do XML
    4.6. Converte PDF para texto
    4.7. Chama OpenAI para responder perguntas
    4.8. Concatena dados do XML com respostas do OpenAI em um dicionário
    4.9. Insere dicionário no MongoDB

PARA ATUALIZAÇÃO SEMANAL (2024):
1. Busca ZIP na CVM
2. Lista arquivos dentro do ZIP
3. Extrai CSV do ZIP para dicionário
4. Compara chave 'Versao' do CSV com MongoDB
    4.1. Se 'Versao' no MongoDB for igual ou superior, pula para a próxima empresa
    4.2. Se 'Versao' no MongoDB for inferior ou inexistente (0):
        4.2.1. Busca ZIP do relatório
        4.2.2. Extrai XML do ZIP do relatório
        4.2.3. Obtém dados do XML
        4.2.4. Ajusta dados do XML
        4.2.5. Extrai PDF do XML
        4.2.6. Converte PDF para texto
        4.2.7. Chama OpenAI para responder perguntas
        4.2.8. Concatena dados do XML com respostas do OpenAI em um dicionário
        4.2.9. Atualiza dicionário no MongoDB
"""

In [ ]:
for registro in dicionario_csv:
    chave_busca = {"Cnpj": registro["Cnpj"], "ID_Documento": registro["ID_Documento"]}
    registro_existente = colecao.find_one(chave_busca, {'_id': 0, 'Versao': 1})
    
    versao_atual = int(registro["Versao"])
    versao_existente = int(registro_existente["Versao"]) if registro_existente else 0
    
    if versao_atual > versao_existente:
        registro_processado = processar_registro(registro)
        colecao.update_one(chave_busca, {"$set": registro_processado}, upsert=True)
        print("Registro processado e atualizado/inserido no banco.")
    else:
        print("Registro ignorado (versão mais recente já está no banco ou versão é igual).")